#### LangChain & Pinecone imports

In [64]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import os
from pinecone import Pinecone, ServerlessSpec  
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain_pinecone import PineconeVectorStore
from langchain_groq import ChatGroq
from langchain import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import ConversationalRetrievalChain

#### Uploading the file 

In [50]:
file_path = ("./Progressive_supranuclear_palsy_diagnosis_and_manag.pdf")

In [ ]:
loader = PyPDFLoader(file_path)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 4)
docs = text_splitter.split_documents(documents)

#### Load HuggingFace Embeddings (used for vector representation of text)

In [53]:
embeddings = HuggingFaceEmbeddings()

C:\Users\User\AppData\Local\Temp\ipykernel_18232\3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


#### Create a new Pinecone index if it doesn't already exist

In [ ]:
os.environ["PINECONE_API_KEY"] = "pcsk_6dFzZs_3taqS6jmyUsFbWp8rXXzqVYd3Sr99Ah91EQLifun6R74wiRdYcgcvALxmqv5bm2"  # Replace with your actual API key

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = 'rag-chatbot'

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768, 
        metric="cosine",  
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
index = pc.Index(index_name)

docsearch = PineconeVectorStore(pinecone_api_key = "pcsk_6dFzZs_3taqS6jmyUsFbWp8rXXzqVYd3Sr99Ah91EQLifun6R74wiRdYcgcvALxmqv5bm2", index_name=index_name, embedding=embeddings)

In [ ]:
# Load the Groq-hosted LLM (LLaMA 3)
llm = ChatGroq(
    model="llama3-8b-8192", 
    api_key = "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"
)


#### Prompt template for RAG with memory

In [63]:
template = """
You are a expert assistant. You are specialized in progressive supranulear palsy. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 2 sentences and concise.

{chat_history}

Context: {context}
Question: {question}
Answer: 
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ["context","chat_histroy","question"]
)

#### Used for tracking history

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

In [ ]:
# RAG chain with LLM, retriever, memory, and custom prompt
rag_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = docsearch.as_retriever(),
    memory = memory,
    combine_docs_chain_kwargs = {"prompt": prompt}
)


In [60]:
question = "What are the early symptoms of progressive supranuclear palsy?"
response = rag_chain.invoke(question)

print("Answer:", response["answer"])
print("\nChat History:")
for msg in response["chat_history"]:
    print(f"{msg.type.capitalize()}: {msg.content}")


Answer: The early symptoms of progressive supranuclear palsy (PSP) typically include subtle changes in gait, balance, and coordination, often characterized by a stiff or awkward walking style, as well as difficulty with swallowing, speech, and eye movements. These symptoms usually progress slowly, often over several months or years, before other cognitive, behavioral, and motor impairments become apparent.

Chat History:
Human: What are the early symptoms of progressive supranuclear palsy?
Ai: The early symptoms of progressive supranuclear palsy (PSP) typically include subtle changes in gait, balance, and coordination, often characterized by a stiff or awkward walking style, as well as difficulty with swallowing, speech, and eye movements. These symptoms usually progress slowly, often over several months or years, before other cognitive, behavioral, and motor impairments become apparent.


In [61]:
question = "how can we prevent PSP is there any preventive measures for it"
response = rag_chain.invoke(question)

print("Answer:", response["answer"])
print("\nChat History:")
for msg in response["chat_history"]:
    print(f"{msg.type.capitalize()}: {msg.content}")

Answer: There are no known preventive measures for progressive supranuclear palsy (PSP), and it is not possible to prevent or delay its onset. PSP is a progressive and degenerative brain disorder, and its exact cause is still unknown, making prevention or delay of onset not currently possible.

Chat History:
Human: What are the early symptoms of progressive supranuclear palsy?
Ai: The early symptoms of progressive supranuclear palsy (PSP) typically include subtle changes in gait, balance, and coordination, often characterized by a stiff or awkward walking style, as well as difficulty with swallowing, speech, and eye movements. These symptoms usually progress slowly, often over several months or years, before other cognitive, behavioral, and motor impairments become apparent.
Human: how can we prevent PSP is there any preventive measures for it
Ai: There are no known preventive measures for progressive supranuclear palsy (PSP), and it is not possible to prevent or delay its onset. PSP i